In [2]:
!pip install transformers
!pip install sentence_splitter
!pip install sentencepiece
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-n

In [10]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sentence_splitter import SentenceSplitter, split_text_into_sentences

In [4]:
#Setup the Transformer NLP model
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print ("device ",device)
model = model.to(device)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
bert_model = SentenceTransformer(model_name)

In [14]:
def SummarizeAndPlagCheck(context):
  splitter = SentenceSplitter(language='en')
  sentence_list = splitter.split(context)
  #print(sentence_list)
  pfh_sent = []
  for i in range (len(sentence_list)):
    context = sentence_list[i]
    text = "paraphrase: "+context + " </s>"
    encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
    input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    model.eval()
    diverse_beam_outputs = model.generate(
        input_ids=input_ids,attention_mask=attention_mask,
        max_length=128,
        early_stopping=True,
        num_beams=5,
        num_beam_groups = 5,
        num_return_sequences=5,
        diversity_penalty = 0.70)
    
    sent = tokenizer.decode(diverse_beam_outputs[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
    pfh_sent.append(sent)
  df = pd.DataFrame([sentence_list, pfh_sent]) # Each list would be added as a row
  df = df.transpose() # To Transpose and make each rows as columns
  df.columns = ['Original', 'Paraphrased'] # Rename the columns
  df.to_csv('Summarized_Original.csv')
  #print (sentence_list, pfh_sent)

In [15]:
context = "Cutting carbon emissions to mitigate climate change and reducing the over-dependence on fossil fuels are listed as a priority in a great number of countries throughout the world. As a result, investments in the energy sector are shifting towards renewable energies; among all the renewables, wind energy is experiencing remarkable growth in recent years. Globally, the wind energy sector has undergone two record years in new capacity installations – a total of 93 GW in 2020 [1] and 94 GW in 2021 [2]. However, these reports also emphasise the need to quadruple the current growth rate to stay on course for the ambitious goals of the Paris Agreement [3] and the European Union Green Deal [4], i.e., net-zero emissions by 2050."
SummarizeAndPlagCheck(context)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
